In [ ]:
import os

fifo_path = "/tmp/myfifo.avi"
os.mkfifo(fifo_path)

In [ ]:
from threading import Thread
import sys
import contextlib

# Create a function that will print under this cell
# even if the main thread is executing in another thread...
# There is not lock, so some misstakes will happen...
this_cell = sys.stdout.parent_header
def cell_print(*args, **kwargs):
    current_cell = sys.stdout.parent_header
    try:
        sys.stdout.parent_header = this_cell
        print(*args, **kwargs)
        sys.stdout.flush()
    finally:
        sys.stdout.parent_header = current_cell

# Create a consumer that will read data from the pipe until
# the writer closes the pipe
def consumer(path):
        cell_print("Wait for producer...")
        with open(path, "rb") as fio:  # Open the pipe
            cell_print("Wait for data...")
            while True:
                data = fio.read(int(1e6))  # Read from the pipe
                cell_print("Read %d bytes.." % len(data))
                if len(data) == 0:
                    cell_print("OK, done.")
                    return
            
th = Thread(target=consumer, args=(fifo_path,))
th.start()

In [ ]:
import cv2
import numpy as np
import itertools
import time
import ipywidgets as widgets

out = widgets.Output(layout={'border': '1px solid black'})
display(out)

video = catalog.load("virat_tiny")

writer = cv2.VideoWriter(
        fifo_path, cv2.VideoWriter_fourcc(*video.fourcc), video.fps, video.size
)

with out:
    for i, frame in enumerate(itertools.cycle(video)):
        out.clear_output()
        print("Write frame %d" % i)
        sys.stdout.flush()
        writer.write(np.asarray(frame))
        if i > 1e4:
            break
        time.sleep(.01)

    print("OK, done.")
    sys.stdout.flush()
writer.release()

In [ ]:
os.remove(fifo_path)